In [ ]:
import joblib
import shap
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt


# Load the saved models


In [ ]:
fraud_model = joblib.load("fraud_random_forest.joblib")
credit_model = joblib.load("credit_random_forest.joblib")


# Load datasets


In [ ]:
fraud_data = pd.read_csv('../data/Fraud_Data.csv')
creditcard_data = pd.read_csv('../data/creditcard.csv')


# Data Preparation
# Fraud Data


In [ ]:
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])
fraud_data['signup_time'] = fraud_data['signup_time'].astype(int) / 10**9
fraud_data['purchase_time'] = fraud_data['purchase_time'].astype(int) / 10**9


X_fraud = fraud_data.drop(columns=['class'])
y_fraud = fraud_data['class']


# Credit Card Data


In [ ]:
X_credit = creditcard_data.drop(columns=['Class'])
y_credit = creditcard_data['Class']


# Train-Test Split


In [ ]:
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)
X_credit_train, X_credit_test, y_credit_train, y_credit_test = train_test_split(X_credit, y_credit, test_size=0.2, random_state=42)


# SHAP for Fraud Data


In [ ]:
explainer = shap.Explainer(fraud_model.named_steps['model'], fraud_model.named_steps['preprocessor'].transform(X_fraud_train))
shap_values = explainer(fraud_model.named_steps['preprocessor'].transform(X_fraud_test))


# Summary Plot


In [ ]:
shap.summary_plot(shap_values, fraud_model.named_steps['preprocessor'].transform(X_fraud_test), feature_names=fraud_model.named_steps['preprocessor'].get_feature_names_out())


# Force Plot


In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1], fraud_model.named_steps['preprocessor'].transform(X_fraud_test)[1], feature_names=fraud_model.named_steps['preprocessor'].get_feature_names_out())


# LIME for Fraud Data


In [ ]:
lime_explainer = lime.lime_tabular.LimeTabularExplainer(X_fraud_train, feature_names=fraud_model.named_steps['preprocessor'].get_feature_names_out(), class_names=['Not Fraud', 'Fraud'], discretize_continuous=True)


# Explain a single prediction


In [ ]:
i = 1
exp = lime_explainer.explain_instance(fraud_model.named_steps['preprocessor'].transform(X_fraud_test)[i], fraud_model.named_steps['model'].predict_proba, num_features=10)
exp.show_in_notebook(show_table=True, show_all=False)